In [2]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import pyttsx3
import datetime
import webbrowser
import wikipedia
import os
import speech_recognition as sr
import cv2
import pytesseract
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier


In [3]:

# Set the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'


In [4]:

# Initialize text-to-speech engine
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)
engine.setProperty('rate', 150)


In [5]:

# Initialize speech recognizer
recognizer = sr.Recognizer()
microphone = sr.Microphone()


In [6]:

# Generate 200 sample texts
sample_texts = [
    "open youtube",
    "open google",
    "play music",
    "open gmail",
    "open whatsapp",
    "tell me the time",
    "tell me the date",
    "search Wikipedia",
    "open camera",
    "stop"
] * 20  # Repeat the list 20 times to get 200 samples

# Generate labels accordingly
labels = [i for i in range(10)] * 20  # Assuming 10 labels, repeated 20 times


In [7]:

# Vectorize the sample texts
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sample_texts)

# Train Random Forest classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X, labels)


RandomForestClassifier(random_state=42)

In [8]:

def speak(audio):
    """Convert text to speech."""
    engine.say(audio)
    engine.runAndWait()

def greet():
    """Greet the user based on the time of the day."""
    hour = int(datetime.datetime.now().hour)
    if hour >= 0 and hour < 12:
        speak("Good Morning!")
    elif hour >= 12 and hour < 18:
        speak("Good Afternoon!")
    else:
        speak("Good Evening!")
    speak("Welcome, I am your personal assistant. How can I assist you today?")

def recognize_speech():
    """Recognize speech using the microphone."""
    with microphone as source:
        speak("I am listening...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    try:
        speak("Recognizing...")
        text = recognizer.recognize_google(audio, language="en-IN")
        print("Recognized Text:", text)  # Print recognized text
        return text.lower()
    except sr.RequestError:
        speak("Sorry, I couldn't reach the Google API.")
    except sr.UnknownValueError:
        speak("Sorry, I could not understand the audio.")
        return ""
    return ""



In [9]:
def perform_action(command):
    """Perform the action based on the command."""
    command_features = vectorizer.transform([command])
    predicted_label = classifier.predict(command_features)[0]
    print("Voice Command:", command)  # Print voice command
    if predicted_label == 0:
        speak("Opening YouTube")
        webbrowser.open("https://www.youtube.com/")
    elif predicted_label == 1:
        speak("Opening Google")
        webbrowser.open("https://www.google.co.in/")
    elif predicted_label == 2:
        speak("Opening music player...")
        os.startfile("C:\\Program Files (x86)\\Windows Media Player\\wmplayer.exe")
    elif predicted_label == 3:
        speak("Opening Gmail")
        webbrowser.open("https://mail.google.com/mail/u/0/#inbox")
    elif predicted_label == 4:
        speak("Opening WhatsApp")
        webbrowser.open("https://web.whatsapp.com/")
    elif predicted_label == 5:
        speak(f"The time is {datetime.datetime.now().strftime('%I:%M %p')}")
    elif predicted_label == 6:
        speak(f"Today's date is {datetime.datetime.now().strftime('%A, %B %d, %Y')}")
    elif predicted_label == 7:
        speak("What do you want to search on Wikipedia?")
        search_query = recognize_speech()
        if search_query:
            speak("Searching Wikipedia...")
            try:
                result = wikipedia.summary(search_query, sentences=2)
                speak("According to Wikipedia,")
                speak(result)
            except wikipedia.exceptions.WikipediaException:
                speak("Sorry, I couldn't find information on that topic.")
    elif predicted_label == 8:
        speak("Opening Camera...")
        open_webcam()
    elif predicted_label == 9:
        speak("Thank you. Bye. Have a nice day.")
        root.destroy()
    else:
        speak("Sorry, I didn't understand that command. Please try again.")


In [10]:

def open_webcam():
    """Capture an image from the webcam and return it."""
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        speak("Could not open webcam.")
        return None
    ret, frame = cap.read()
    cap.release()
    if ret:
        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        img.show()  # Display the captured image
        text = extract_text_from_image(img)
        if text:
            speak(f"The extracted text is: {text}")
        else:
            speak("No text found in the image.")
        print("Extracted Text:", text)  # Print the extracted text for debugging

def extract_text_from_image(img):
    """Extract text from an image using OCR."""
    gray = img.convert('L')  # Convert image to grayscale
    text = pytesseract.image_to_string(gray)
    return text

def on_voice_command_button():
    greet()  # Greet the user before listening for commands
    command = recognize_speech()
    if command:
        update_text_label(command)  # Update label with recognized text
        perform_action(command)
    else:
        speak("I didn't catch that. Could you please repeat?")

def create_big_circle_button(root):
    circle_button = tk.Button(root, text="Speak", bg="blue", fg="white", font=("Helvetica", 24), command=on_voice_command_button)
    circle_button.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

def on_upload_button():
    filepath = filedialog.askopenfilename(filetypes=[("Image Files", ".png;.jpg;.jpeg;.bmp")])
    if filepath:
        img = Image.open(filepath)
        img.show()  # Display the uploaded image
        text = extract_text_from_image(img)
        if text:
            speak(f"The extracted text is: {text}")
        else:
            speak("No text found in the image.")
        print("Extracted Text:", text)  # Print the extracted text for debugging

def on_webcam_button():
    open_webcam()

def update_text_label(text):
    """Update the text label with the recognized text."""
    text_label.config(text=f"Recognized Text: {text}")


In [11]:

root = tk.Tk()
root.title("Smart text reader")
root.geometry("400x300")

text_label = tk.Label(root, text="", font=("Helvetica", 12))
text_label.pack()

upload_button = tk.Button(root, text="Upload Image", command=on_upload_button)
upload_button.pack(pady=5)

webcam_button = tk.Button(root, text="Open Webcam", command=on_webcam_button)
webcam_button.pack(pady=5)

create_big_circle_button(root)

root.mainloop()
